# Primer ejemplo con Keras + Python

Este es nuestro primer [Jupyter Notebook](http://jupyter.org) de Deep Learning con [Keras](https://keras.io) (y Python). 

En este primer notebook únicamente mostraremos un ejemplo de red neuronal con un par de capas que resuelve el famoso problema MNIST. El objetivo es simplemente ver el flujo de trabajo habitual para atacar un problema con Keras, pero no nos detendremos en los detalles de cómo se usa Keras ni de la potencia que podemos extraer a las diversas funcionalidades que proporciona. En notebooks posteriores profundizaremos en su uso y en las características propias de Deep Learning que podemos abordar.

Para ejecutar los diversos chunks (así se llaman) de código que conforman este notebook puedes pulsar en el botón *Run* que puedes encontrar en cada uno de ellos, o bien situando el cursor dentro del chunk y presionando la comnbinación *Ctrl+Shift+Enter* (también tienes la opción de usar las opciones de ejecución en el menú *Run* de la barra del navegador Jupyter. Ten en cuenta que algunas opciones pueden cambiar dependiendo del entorno en el que estés trabajando con este notebook (Jupyter Notebook, Jupyter Lab, o Collab de Google.. cualquiera de los tres funcionan de forma similar, pero con ligeras diferencias). 

## Nuestro primer modelo con Keras

El primer paso es cargar la librería keras que permitirá interactuar a Python con la librería de Deep Learning que usemos (en nuestro caso, [Tensorflow](https://www.tensorflow.org)).

In [1]:
import keras
keras.__version__
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2214218177018818445
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1525841100
locality {
  bus_id: 1
  links {
  }
}
incarnation: 908214438215550088
physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


### Preparación de los datos

A continuación vamos a cargar los datos del problema [MNIST](https://en.wikipedia.org/wiki/MNIST_database), una gran base de datos de dígitos escritos a mano que sevirá como primera aproximación a la resolución de un problema de clasificación haciendo uso de Redes Neuronales.

Afortunadamente, este problema es tan común que Keras proporciona una instrucción directa para descargar las imágenes (de 28x28 pixels en escala de grises) que representan los miles de dígitos escritos a mano. El paso, que veremos que es habitual en muchos problemas que veremos en el curso, consta de dos pasos: primero cargar la librería de Keras que prporciona las herramientas para trabajar con el dataset concreto (que comúnmente están en el paquete `keras.datasets`, y después ejecutar el proceso de carga de los datos (que serán descargados la primera vez desde un repositorio que viene por defecto predefinido en ese paquete):

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Observa que el proceso de carga de datos separa adecuadamente las diversas partes de que consta este dataset:  (conjunto de entrenamiento, conjunto de test), y cada uno de estos conjuntos está formado por un conjunto de imágenes, con sus respectivas etiquetas de clasificación (*labels*).

Podemos explorar un poco cómo son cada una de estas variables haciendo uso de instrucciones específicas de Python que nos dan información acerca de su estructura y muestra los primeros valores:

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
len(train_labels)

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

De forma análoga, podemos explorar las imágenes que se usarán para test:

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
len(test_labels)

10000

In [8]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Si queremos ver alguna de las imágenes que hay en el dataset, podemos hacer uso de la instrucción adecuada de, por ejemplo, la librería `matplotlib`:

In [ ]:
from matplotlib import pyplot as plt
def gen_image(arr):
    two_d = (np.reshape(arr, (28, 28)) * 255).astype(np.uint8)
    plt.imshow(two_d, interpolation='nearest')
    return plt

# Get a batch of two random images and show in a pop-up window.
batch_xs, batch_ys = mnist.test.next_batch(2)
gen_image(batch_xs[0]).show()
gen_image(batch_xs[1]).show()

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The 
network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we 
will verify if these predictions match the labels from `test_labels`.

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

In [9]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))


The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some 
data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully 
representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers 
which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a 
succession of increasingly refined data filters -- the "layers".

Here our network consists of a sequence of two `Dense` layers, which are densely-connected (also called "fully-connected") neural layers. 
The second (and last) layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each 
score will be the probability that the current digit image belongs to one of our 10 digit classes.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be 
able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly 
classified).

The exact purpose of the loss function and the optimizer will be made clear throughout the next two chapters.

In [10]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in 
the `[0, 1]` interval. Previously, our training images for instance were stored in an array of shape `(60000, 28, 28)` of type `uint8` with 
values in the `[0, 255]` interval. We transform it into a `float32` array of shape `(60000, 28 * 28)` with values between 0 and 1.

In [11]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

We also need to categorically encode the labels, a step which we explain in chapter 3:

In [12]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

We are now ready to train our network, which in Keras is done via a call to the `fit` method of the network: 
we "fit" the model to its training data.

In [13]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 4s 72us/step - loss: 0.2568 - acc: 0.9260
Epoch 2/5
60000/60000 [==============================] - 4s 64us/step - loss: 0.1032 - acc: 0.9698
Epoch 3/5
60000/60000 [==============================] - 4s 65us/step - loss: 0.0678 - acc: 0.9800
Epoch 4/5
60000/60000 [==============================] - 4s 68us/step - loss: 0.0496 - acc: 0.9849
Epoch 5/5
60000/60000 [==============================] - 4s 64us/step - loss: 0.0371 - acc: 0.9892


Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over 
the training data.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data. Now let's check that our model performs well on the test set too:

In [14]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 65us/step


In [15]:
print('test_acc:', test_acc)

test_acc: 0.9784



Our test set accuracy turns out to be 97.8% -- that's quite a bit lower than the training set accuracy. 
This gap between training accuracy and test accuracy is an example of "overfitting", 
the fact that machine learning models tend to perform worse on new data than on their training data. 
Overfitting will be a central topic in chapter 3.

This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits, in 
less than 20 lines of Python code. In the next chapter, we will go in detail over every moving piece we just previewed, and clarify what is really 
going on behind the scenes. You will learn about "tensors", the data-storing objects going into the network, about tensor operations, which 
layers are made of, and about gradient descent, which allows our network to learn from its training examples.